## Final project

In [1]:
import facebook_scraper as fs
import pandas as pd
import datetime
import numpy as np
import os
import time
from facebook_scraper import _scraper
from pprint import pprint

/home/vnn04/miniconda3/envs/ml-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
FANPAGE_LINK = "sieuanhhungMarvel"
current_dir = os.getcwd()
FOLDER_PATH = current_dir + "/Data/"
print(FOLDER_PATH)

/home/vnn04/Documents/facebook_scraper_page/Data/


#### Nhập tài khoản và mật khẩu facebook

In [3]:
user_email = "0814277945"
user_password = "hithere123"

#### Set cookies

In [4]:
_scraper.login(user_email, user_password)
cookies = _scraper.session.cookies
# Store & retrieve cookies
fs.set_cookies(cookies)
# Check it worked
print(_scraper.is_logged_in())

/home/vnn04/miniconda3/envs/ml-env/lib/python3.12/site-packages/facebook_scraper/facebook_scraper.py:855: UserWarning: Facebook language detected as vi_VN - for best results, set to en_US
  warnings.warn(


True


In [5]:
day_start = datetime.datetime(2023, 11, 19, 0, 0, 0)
day_end = datetime.datetime(2023, 9, 19, 0, 0, 0)

#### Lấy thông tin của các bài viết trong 2 tháng vừa qua (từ 19/9/2023 - 19/11/2023)

In [14]:
post_list = []

for post in fs.get_posts(FANPAGE_LINK,
                    options={"comments": False, "reactions": False, "allow_extra_requests": False},
                    pages=1000):
    if post['time'] > day_start:
        continue
    else:
        if post['time'] <= day_end:
            break
        print(post)
        post_list.append({'post_id':post['post_id'], 'text':post['text'], 'time':post['time'], 'images_low':post['images_lowquality']})


{'post_id': '734518898719448', 'text': 'Các bạn nam chắc ai cũng muốn học chiêu pause time này để dành khi cãi nhau với người yêu :))', 'post_text': 'Các bạn nam chắc ai cũng muốn học chiêu pause time này để dành khi cãi nhau với người yêu :))', 'shared_text': '', 'original_text': None, 'time': datetime.datetime(2023, 11, 18, 22, 40, 26), 'timestamp': 1700322026, 'image': None, 'image_lowquality': 'https://scontent.fhan14-4.fna.fbcdn.net/v/t39.30808-6/402264624_734518948719443_5938829508463439003_n.jpg?stp=cp0_dst-jpg_e15_p160x160_q65&_nc_cat=103&ccb=1-7&_nc_sid=5f2048&efg=eyJpIjoidCJ9&_nc_ohc=NRb9YeaNfLQAX87bQAf&_nc_ht=scontent.fhan14-4.fna&oh=00_AfDAQ3TeMsdn-2JYe1MPu7zBzxEweBWhv06gBlsC86b5aA&oe=6560B153', 'images': None, 'images_description': None, 'images_lowquality': ['https://scontent.fhan14-4.fna.fbcdn.net/v/t39.30808-6/402264624_734518948719443_5938829508463439003_n.jpg?stp=cp0_dst-jpg_e15_p160x160_q65&_nc_cat=103&ccb=1-7&_nc_sid=5f2048&efg=eyJpIjoidCJ9&_nc_ohc=NRb9YeaNfLQAX87bQ

#### Số lượng post lấy được từ 19/9/2023 đến 19/11/2023

In [17]:
print(len(post_list))

216


#### Lưu danh sách thông tin cơ bản của các post vào file sieuanhhungMarvel.csv

In [18]:
import csv

path = FOLDER_PATH + FANPAGE_LINK + ".csv"

with open(path, 'w', newline='', encoding='utf-8') as csv_file:
    # Tạo đối tượng csv.writer
    csv_writer = csv.writer(csv_file)

    # Ghi header
    header = post_list[0].keys()
    csv_writer.writerow(header)

    # Ghi dữ liệu
    for row in post_list:
        csv_writer.writerow(row.values())


#### Đọc post_id từ file sieuanhhungMarvel.csv

In [13]:
path = FOLDER_PATH + FANPAGE_LINK + ".csv"

In [14]:
df_post = pd.read_csv(path)

df_post_id = df_post['post_id']
df_post_id.head(10)

0    734518898719448
1    734285772076094
2    734234215414583
3    734194055418599
4    733990162105655
5    733860328785305
6    733772145460790
7    733681738803164
8    733428062161865
9    733305832174088
Name: post_id, dtype: int64

#### Lấy tất cả id của các post cần crawl dữ liệu

In [15]:
post_id = df_post_id.to_numpy()

#### Lấy reactions, comments, shares, comments_full

In [5]:
def get_reactions_comments_full_shares(url):
    info = []
    for data in fs.get_posts(post_urls=[url], options={'reactions':True, 'comments':True}, cookies='cookies.txt'):
        info.append({'reactions':data['reactions'], 'reaction_count':data['reaction_count'], 'shares':data['shares'], 'comments':data['comments'], 'comments_full':data['comments_full']})
    return info

def get_more_info(url):
    data = get_reactions_comments_full_shares(url)

    if data is None or not data:
        return 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

    if 'reactions' == None:
        like, love, haha, sad, wow, care, angry = 0, 0, 0, 0, 0, 0, 0
    else:
        reactions = data[0]['reactions']
        if 'like' not in reactions:
            like = 0
        else:
            like = reactions['like']
        if 'love' not in reactions:
            love = 0
        else:
            love = reactions['love']
        if 'haha' not in reactions:
            haha = 0
        else:
            haha = reactions['haha']
        if 'wow' not in reactions:
            wow = 0
        else:
            wow = reactions['wow']
        if 'sad' not in reactions:
            sad = 0
        else:
            sad = reactions['sad']
        if 'care' not in reactions:
            care = 0
        else:
            care = reactions['care']
        if 'angry' not in reactions:
            angry = 0
        else:
            angry = reactions['angry']

    reaction_count = data[0].get('reaction_count', 0)
    shares = data[0].get('shares', 0)
    comment = data[0].get('comments', 0)
    comments_full = data[0].get('comments_full', [])

    return {'post_id':url, 'like':like, 'love':love, 'haha':haha, 'sad':sad, 'wow':wow, 'care':care, 'angry':angry, 'reaction_count':reaction_count, 'shares':shares, 'comment':comment, 'comments_full':comments_full}


#### Thêm cột like, love, haha, care, sad, angry, reaction_count, comments, comments_full, shares

In [ ]:
result = []
for i in range(0, len(post_id)):
    print(i)
    more_info = get_more_info(post_id[i])
    result.append(more_info)

##### Lưu dữ liệu vừa lấy vào file temp

In [62]:
path = FOLDER_PATH + 'temp' + ".csv"

df_temp = pd.DataFrame(result, columns=["like","love","haha","sad","wow","care","angry","reaction_count","shares","comment","comments_full"])
df_temp.to_csv(path, index=False)

#### Gộp vào file sieuanhhungMarvel

In [ ]:
df_result = pd.merge(df_post, df_temp, on='post_id', how='inner')

#### Lưu dữ liệu vào file sieuanhhungMarvel.csv

In [ ]:
df_result.to_csv(FOLDER_PATH + "sieuanhhungMarvel.csv", index=False)

#### Lấy reactors của post có nhiều reactions nhất

In [5]:
reactors_list = []

for reactor in fs.get_reactors(post_id='726632232841448'):
    print(reactor)
    reactors_list.append(reactor)
    time.sleep(0.2)

{'name': 'Phan Thanh Tùng', 'link': 'https://facebook.com/profile.php?id=100095605751322&eav=AfZM_Cg-nI7lLJFb2hOtIlb10miqZyQBrFuVd4c_Kj2Vy1YdOM7ezAxIvROhCJaNESA&fref=pb&paipv=0', 'type': 'haha'}
{'name': 'Diem Quynh', 'link': 'https://facebook.com/profile.php?id=100095605181131&eav=AfbKfV8MWc9ZpuBGz1GM0EPu86wD-8trmEw8H5ASlZ4FXchinY_PQ9edlCzGBhqvlrk&fref=pb&paipv=0', 'type': 'haha'}
{'name': 'Nguyễn Thị Xuân Trâm', 'link': 'https://facebook.com/profile.php?id=100095579735068&eav=AfYiZ8alOxgwWARo9Un77RxeJufhUJpsKN04JRMJ-FG1rTPocOE_ZfdaUEOsR-e5_2g&fref=pb&paipv=0', 'type': 'haha'}
{'name': 'Woo Kyu', 'link': 'https://facebook.com/profile.php?id=100095559191414&eav=AfZxFfI6GSDQZZSKBmpBLfGuMAO6neNSsx9RPrk8GyCNlMlsUaslRDnacMjUrRKQo6c&fref=pb&paipv=0', 'type': 'like'}
{'name': 'Area Luna', 'link': 'https://facebook.com/profile.php?id=100095513197010&eav=AfajJhDqFP1XjrzyVq1WcvPudy8e2FiO_H1woU8jNN6bJn6VemiumyJhs3VKO-fnffg&fref=pb&paipv=0', 'type': 'love'}
{'name': 'Phuong Lee', 'link': 'https:/

In [18]:
import csv
path = FOLDER_PATH + "reactos" + ".csv"

with open(path, 'w', newline='', encoding='utf-8') as csv_file:
    # Tạo đối tượng csv.writer
    csv_writer = csv.writer(csv_file)

    # Ghi header
    header = reactors_list[0].keys()
    csv_writer.writerow(header)

    # Ghi dữ liệu
    for row in reactors_list:
        csv_writer.writerow(row.values())

#### Lấy thông tin cơ bản của người thả reaction (id, tên, ngày sinh, giới tính, quê quán)

In [5]:
import facebook

def get_user_info(user_id):
    access_token = 'EAABwzLixnjYBOx7UeqsdZByQ59eH1SljdWTU2lvwvA7gT9cX87hrYHyCehkLFyEY5YTxqEy5AaT1NzJbAcfZA95FaJGItTOqUlZCK2HaSZAq7lIvNRUQ8Bho3B24u2k2v06EcieGpC9QZAVV9vKMoRxwxxGAtZC9jetgQbfzLIOsmrEIZBN0VMB8n6khiInUo2dqnsZD'

    fields = 'id,name,birthday,gender,hometown,location'

    graph = facebook.GraphAPI(access_token)
    try:
        user_info = graph.get_object(user_id, fields=fields)
    except facebook.GraphAPIError as e:
        user_info = None
        print(f"Error: {e}")
    time.sleep(1)
    return user_info

In [3]:
reactors_df = pd.read_csv('Data/reactos.csv')
ids = reactors_df['user_id'].tolist()

reactors_info = []

In [6]:
for i in range(1800, len(ids)):
    print(i)
    reactor_info = get_user_info(str(ids[i]))
    print(reactor_info)
    reactors_info.append(reactor_info)
    time.sleep(1)

1800
{'id': '100078794572894', 'name': 'Nguyễn Thị Hồng Quyên', 'gender': 'female'}
1801
{'id': '100078788474155', 'name': 'Trâm Võ', 'gender': 'female', 'hometown': {'id': '108663825829925', 'name': 'Chaudok, An Giang, Vietnam'}, 'location': {'id': '108663825829925', 'name': 'Chaudok, An Giang, Vietnam'}}
1802
{'id': '100078787875586', 'name': 'Yen Luong', 'gender': 'female'}
1803
{'id': '100078785754166', 'name': 'Phuong Uyen', 'birthday': '08/04/2004', 'gender': 'female'}
1804
{'id': '100078780517611', 'name': 'Ng Minhh Phuong', 'gender': 'female'}
1805
Error: Unsupported get request. Object with ID 'thhuen' does not exist, cannot be loaded due to missing permissions, or does not support this operation. Please read the Graph API documentation at https://developers.facebook.com/docs/graph-api
None
1806
{'id': '100078767682985', 'name': 'Nanh Thuhai', 'gender': 'male'}
1807
{'id': '100078765926609', 'name': 'Nguyễn Thuỵ Gia Linh', 'gender': 'female', 'location': {'id': '10798335256891

/home/vnn04/miniconda3/envs/ml-env/lib/python3.12/site-packages/facebook/__init__.py:99: SyntaxWarning: invalid escape sequence '\d'
  version_regex = re.compile("^\d\.\d{1,2}$")


KeyboardInterrupt: 

In [7]:
for i in range(len(reactors_info)):
    print(reactors_info[i])

None
None
None
None
{'id': '100078794572894', 'name': 'Nguyễn Thị Hồng Quyên', 'gender': 'female'}
{'id': '100078788474155', 'name': 'Trâm Võ', 'gender': 'female', 'hometown': {'id': '108663825829925', 'name': 'Chaudok, An Giang, Vietnam'}, 'location': {'id': '108663825829925', 'name': 'Chaudok, An Giang, Vietnam'}}
{'id': '100078787875586', 'name': 'Yen Luong', 'gender': 'female'}
{'id': '100078785754166', 'name': 'Phuong Uyen', 'birthday': '08/04/2004', 'gender': 'female'}
{'id': '100078780517611', 'name': 'Ng Minhh Phuong', 'gender': 'female'}
None
{'id': '100078767682985', 'name': 'Nanh Thuhai', 'gender': 'male'}
{'id': '100078765926609', 'name': 'Nguyễn Thuỵ Gia Linh', 'gender': 'female', 'location': {'id': '107983352568918', 'name': 'Bắc Ninh'}}
None
{'id': '100078737587573', 'name': 'Như Ýy', 'gender': 'female'}
{'id': '100078717253698', 'name': 'Soo Jin', 'gender': 'female'}
{'id': '100078710365785', 'name': 'Ngọc Thi', 'gender': 'female', 'hometown': {'id': '115357638478734', 

In [ ]:
temp = reactors_info
new_list = []
for item in temp:
    if item is not None:
        new_list.append(item)

new_df = pd.DataFrame(new_list)
new_df.to_csv('temp1.csv', index=False)

In [ ]:
len(reactors_info)

1076

In [67]:
print(user_info)

{'Friend_count': None, 'Follower_count': None, 'Following_count': None, 'id': '100039693473055', 'Name': 'Nguyễn V. Thân', 'Education': {'University': 'Trường Đại học Công nghệ - Đại học Quốc gia Hà Nội', 'THPT Thanh Miện I': 'September 2022 - Present', 'School year 2022': 'High School'}, 'Places lived': [{'link': '/profile.php?id=103934556308311&eav=Afb2B5OB4j9nzSEjs9Gr95dgGCMc6_EZclNCMAVDfh96wkuWJ2eWaoBaY0_UL417jBk&paipv=0', 'text': 'Hải Dương', 'type': 'Current town/city'}], 'Contact Info': {'Facebook': '/vanthan.511'}, 'Life events': ''}


In [13]:
df = pd.read_csv('temp0.csv')
df1 = pd.read_csv('temp.csv')

new_df = pd.concat([df, df1])
new_df

,id,name,gender,hometown,location,birthday
0,100095605751322,Phan Thanh Tùng,female,NaN,NaN,NaN
1,100095605181131,Diem Quynh,female,NaN,NaN,NaN
2,100095579735068,Nguyễn Thị Xuân Trâm,NaN,NaN,NaN,NaN
3,100095559191414,Woo Kyu,NaN,NaN,NaN,NaN
4,100095513197010,Area Luna,female,NaN,NaN,NaN
...,...,...,...,...,...,...
868,100078000098322,Thơm Võ,female,NaN,NaN,NaN
869,100077999779922,Minh Nhật,male,NaN,NaN,08/02
870,100077994279903,Nguyễn Phương Thanh Trúc,female,"{'id': '108458769184495', 'name': 'Ho Chi Minh...","{'id': '108458769184495', 'name': 'Ho Chi Minh...",NaN
871,100077978411013,Diễm Ty,NaN,NaN,NaN,NaN
